# Patch Classifier Query
We are now running both a pixel and patch classifier on Descartes. This notebook takes in a set of patch classifier outputs along with a set of pixel classifier candidates. This notebook finds the intersection between candidate points and patch classifier predictions above a given threshold.

In [ ]:
import geopandas as gpd
# pygeos is required to use geopandas spatial indexing
gpd.options.use_pygeos = True

In [ ]:
region_name = 'nigeria'

In [ ]:
patch_model_version = '3.0'
patch_classifier_fname= f'{region_name}_patch_weak_labels_3.0_2019-01-01_2021-06-01_stride_8'
patch = gpd.read_file(f'../data/model_outputs/patch_classifier/{patch_classifier_fname}.geojson')

In [ ]:
pixel_model_version = '0.0.11'
threshold = 0.6
min_sigma = 5.0
pixel_classifier_fname = f'{region_name}_v0.0.11_2019-01-01_2021-06-01mosaic-median_blobs_thresh_{threshold}_min-sigma_{min_sigma}}_area-thresh_0.0025'
pixel = gpd.read_file(f'../data/model_outputs/candidate_sites/{pixel_model_version}/{pixel_classifier_fname}.geojson')

In [55]:
threshold = 0.3
patch_threshold = patch[patch['mean'] > threshold]
patch_index = patch_threshold['geometry'].sindex

In [56]:
overlap = []
for candidate in pixel['geometry']:
    if len(patch_index.query(candidate)) > 0:
        overlap.append(True)
    else:
        overlap.append(False)
union = pixel[overlap]
print(f"{len(union)} ({len(union) / len(pixel):.0%}) candidate points intersect with patch classifier predictions greater than {threshold}")
union

1003 (28%) candidate points intersect with patch classifier predictions greater than 0.3


,lon,lat,name,geometry
2,8.245198,4.631247,nigeria_v0.0.11_2019-01-01_2021-06-01mosaic-me...,POINT (8.24520 4.63125)
4,3.461528,7.314770,nigeria_v0.0.11_2019-01-01_2021-06-01mosaic-me...,POINT (3.46153 7.31477)
8,7.555160,5.893330,nigeria_v0.0.11_2019-01-01_2021-06-01mosaic-me...,POINT (7.55516 5.89333)
14,6.027849,5.517585,nigeria_v0.0.11_2019-01-01_2021-06-01mosaic-me...,POINT (6.02785 5.51759)
15,8.155129,6.190998,nigeria_v0.0.11_2019-01-01_2021-06-01mosaic-me...,POINT (8.15513 6.19100)
...,...,...,...,...
3510,7.664289,5.969319,nigeria_v0.0.11_2019-01-01_2021-06-01mosaic-me...,POINT (7.66429 5.96932)
3512,8.520512,7.210007,nigeria_v0.0.11_2019-01-01_2021-06-01mosaic-me...,POINT (8.52051 7.21001)
3521,7.923548,6.401040,nigeria_v0.0.11_2019-01-01_2021-06-01mosaic-me...,POINT (7.92355 6.40104)
3522,3.854631,7.263169,nigeria_v0.0.11_2019-01-01_2021-06-01mosaic-me...,POINT (3.85463 7.26317)


In [57]:
file_path = f'../data/model_outputs/candidate_sites/{pixel_model_version}/{pixel_classifier_fname}_patch_v{patch_model_version}_threshold_{threshold}_intersection.geojson'
union.to_file(file_path, driver='GeoJSON')
print("Saved to", file_path)

Saved to ../data/model_outputs/candidate_sites/0.0.11/nigeria_v0.0.11_2019-01-01_2021-06-01mosaic-median_blobs_thresh_0.6_min-sigma_3.5_area-thresh_0.0025_patch_v3.0_threshold_0.3_intersection.geojson
